In [173]:
# import needed libraries 
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [174]:
# import nfl_data_py for direct stream to data
import nfl_data_py as nfl
from nfl_data_py import import_schedules

# Define suffix mapping for merges
MERGE_SUFFIXES = {
    'opp_def':'_opp',           # def stats
    'schedule': '_sched',       # schedule-based info
    'season_avg': '_season',    # league-season averages
    'team_results': '_res'      # team results & streak
}

# Define potential team name issues
TEAM_FIX = {'WSH':'WAS', 'JAC':'JAX'} # Add here if more discovered

In [175]:
# import weekly nfl data for 2021 to 2023
df = nfl.import_weekly_data(years=[2021,2022,2023])
schedules = import_schedules([2021, 2022, 2023])

# Create QB dataset for future use and testing
keep_cols_qbs = [
    'player_id', 'player_name', 'season', 'season_type','week', 'recent_team', 'opponent_team', 'passing_yards', 'attempts',
    'completions', 'passing_tds', 'interceptions', 'rushing_yards', 'rushing_tds', 'fantasy_points', 'fantasy_points_ppr', 'sacks',
]
qbs = df[df['position'] == 'QB']
qbs = qbs[keep_cols_qbs]

# Potential team name fix
qbs['recent_team'] = qbs['recent_team']. replace(TEAM_FIX)
qbs['opponent_team'] = qbs['opponent_team'].replace(TEAM_FIX)
schedules['home_team'] = schedules['home_team'].replace(TEAM_FIX)
schedules['away_team'] = schedules['away_team'].replace(TEAM_FIX)

#print(qbs.head(10))
#print(schedules.head(10))

Downcasting floats.


In [176]:
'''# Validate data is clean (i.e. no dupes, missing values)
print(df.isna().sum())
print(df.dtypes)
print(df.groupby(['season','week','player_name']).size())'''
cols_to_drop = [c for c in qbs.columns if 'home_team' in c or 'away_team' in c or 'home_game' in c]
qbs = qbs.drop(columns=cols_to_drop, errors='ignore')

In [ ]:
# Create helper function to designate home/away flag 
def add_home_away_flags(pos_df, schedules):
    # Simplify schedules to unique team-week mapping
    home_flags = schedules[['season', 'week', 'home_team']].copy()
    home_flags['home_game'] = 1
    home_flags.rename(columns={'home_team': 'team'}, inplace=True)

    away_flags = schedules[['season', 'week', 'away_team']].copy()
    away_flags['home_game'] = 0
    away_flags.rename(columns={'away_team': 'team'}, inplace=True)

    # Combine into one clean mapping
    team_week_flags = pd.concat([home_flags, away_flags], ignore_index=True)

    # Merge safely on team identity
    merged = pos_df.merge(
        team_week_flags,
        left_on=['season', 'week', 'recent_team'],
        right_on=['season', 'week', 'team'],
        how='left'
    )

    merged.drop(columns=['team'], inplace=True, errors='ignore')

    # Ensure the column exists even for missing merges
    if 'home_game' not in merged.columns:
        merged['home_game'] = np.nan
    return merged

# Fix below by recalculating win streak without cumsum

# Create helper function to create win/loss flags and win(loss) streak
def add_team_win_streaks(pos_df, schedules):
    sch = schedules[['season', 'season_type', 'week', 'home_team', 'away_team', 'home_score', 'away_score']].copy()

    # Create win/loss flags
    schedules['home_win'] = np.where(schedules['home_score'] > schedules['away_score'], 1, 0)
    schedules['away_win'] = np.where(schedules['away_score'] > schedules['home_score'], 1, 0)

    # Create results for home/away teams
    home_results = schedules[['season', 'week', 'home_team', 'home_win']].rename(
        columns = {'home_team': 'team', 'home_win': 'team_win'}
    )

    away_results = schedules[['season', 'week', 'away_team', 'away_win']].rename(
        columns = {'away_team': 'team', 'away_win': 'team_win'}
    )

    # Combine both and sort
    team_results = pd.concat([home_results, away_results], ignore_index=True)
    team_results = team_results.sort_values(['team', 'season', 'week'])

    # Calculate streak
    def compute_streaks(x):
        # Reset streak after a loss
        streak = (x['team_win'] * (x['team_win'].groupby((x['team_win'] == 0).cumsum()).cumcount() + 1))
        return streak
    
    team_results['team_win_streak'] = team_results.groupby('team', group_keys=False).apply(compute_streaks)

    # Merge streaks back into positional df
    pos_df = pos_df.merge(
        team_results[['season', 'week', 'team', 'team_win_streak']],
        left_on = ['season', 'week', 'recent_team'],
        right_on = ['season', 'week', 'team'],
        how='left'
    )

    pos_df.drop(columns=['team'], inplace=True, errors='ignore')
    return pos_df

# Create final safe merge for all team context metrics
def merge_team_context(pos_df, schedules):
    # Win/loss flag creation
    schedules = schedules.copy()
    schedules['home_win'] = (schedules['home_score'] > schedules['away_score']).astype(int)
    schedules['away_win'] = (schedules['away_score'] > schedules['home_score']).astype(int)

    # Team-level view for both home and away team
    home_games = schedules[['season', 'week', 'home_team', 'away_team', 'home_win']].rename(
        columns={'home_team': 'team', 'away_team': 'opponent_team', 'home_win':'team_win'}
    )
    home_games['home_game'] = 1

    away_games = schedules[['season', 'week', 'home_team', 'away_team', 'away_win']].rename(
        columns={'away_team': 'team', 'home_team': 'opponent_team', 'away_win':'team_win'}
    )
    away_games['home_game'] = 0

    # Combine into unified "team_games"
    team_games = pd.concat([home_games, away_games], ignore_index=True)
    
    team_games = team_games.dropna(subset=['team', 'opponent_team'])
    team_games.drop_duplicates(subset=['season', 'week', 'team'], inplace=True)

    # Merge
    merged = pos_df.merge(
        team_games,
        left_on = ['season', 'week', 'recent_team', 'opponent_team'],
        right_on = ['season', 'week', 'team', 'opponent_team'],
        how='left'
    )

    # Clean
    merged.drop(columns=['team'], inplace=True, errors='ignore')
    merged = merged.drop_duplicates(subset = ['player_id', 'season', 'week'])
    return merged

In [178]:
# Create predictive metrics for QB

# First sort data to ensure accuracy
qbs = qbs.sort_values(['player_id','season','week']).reset_index(drop=True)
''' Validate it looks right if wanted
qbs.head(10) '''

# Create rolling stats for meaningful OFF metrics
# Completion percentage and efficiency metrics
qbs['completion_pct'] = qbs['completions'] / qbs['attempts'].replace(0, np.nan)
qbs['yards_per_attempt'] = qbs['passing_yards'] / qbs['attempts'].replace(0, np.nan)

# Rolling mean and standard deviation calculated
for stat in ['passing_yards', 'attempts', 'passing_tds', 'interceptions', 'completion_pct', 'yards_per_attempt']:
    qbs[f'{stat}_roll3_mean'] = qbs.groupby('player_id')[stat].transform(lambda x: x.shift().rolling(3).mean())
    qbs[f'{stat}_roll3_std'] = qbs.groupby('player_id')[stat].transform(lambda x: x.shift().rolling(3).std())

# Create average stats for meaningful DEF metrics
# Defense stat generation
opp_def_stats = (
    qbs.groupby(['season', 'opponent_team'])
        .agg(
            opp_avg_pass_yards_allowed=('passing_yards', 'mean'),
            opp_avg_ints_forced=('interceptions', 'mean'),
            opp_avg_sacks=('sacks', 'mean')
        )
        .reset_index()
)

#print(opp_def_stats.columns)
#print(opp_def_stats.head())

# Merge defensive stats to the QB dataset
qbs = qbs.merge(opp_def_stats, on=['season', 'opponent_team'], how='left', suffixes=('', MERGE_SUFFIXES['opp_def']))

#print(qbs[['opponent_team', 'opp_avg_pass_yards_allowed']].head())

# Normalize DEF metrics in respect to league averages in season
season_avgs = qbs.groupby('season')[['passing_yards', 'passing_tds', 'interceptions']].mean().reset_index()
season_avgs.rename(columns={
    'passing_yards': f'passing_yards{MERGE_SUFFIXES["season_avg"]}',
    'passing_tds': f'passing_tds{MERGE_SUFFIXES["season_avg"]}',
    'interceptions': f'interceptions{MERGE_SUFFIXES["season_avg"]}'
}, inplace=True)
qbs = qbs.merge(season_avgs, on='season', how='left')
qbs['opp_pass_efficiency_index'] = qbs['opp_avg_pass_yards_allowed'] / qbs[f'passing_yards{MERGE_SUFFIXES["season_avg"]}']

# Base merge that ensures correct game-to-player mapping
qbs = merge_team_context(qbs, schedules)

# Create game context and momentum metrics
# Add home/away flags
 # Redudancy qbs = add_home_away_flags(qbs, schedules)

# Add win/loss/streak flags
qbs = add_team_win_streaks(qbs, schedules)

qbs['team_won_last'] = qbs['team_win']

#print(qbs[['player_id', 'season', 'week']].duplicated().sum())

# Bye week metric
qbs['prev_week'] = qbs.groupby('player_id')['week'].shift()
qbs['bye_last_week'] = np.where(qbs['week'] - qbs['prev_week'] > 1, 1, 0)
qbs.drop(columns='prev_week', inplace=True)

# Seasonal/Temporal context metrics
qbs['season_week'] = qbs['week']
qbs['is_playoffs'] = np.where(qbs['season_type'] == 'POST', 1, 0)

# Momentum indicators
qbs['passing_yards_diff_last'] = qbs['passing_yards'] - qbs['passing_yards_roll3_mean']
qbs['passing_yards_volatility'] = qbs['passing_yards_roll3_std']
qbs['passing_trend_3'] = qbs.groupby('player_id')['passing_yards_roll3_mean'].diff()
qbs['completion_trend_3'] = qbs.groupby('player_id')['completion_pct_roll3_mean'].diff()

print(qbs.shape)
print(qbs[['player_id', 'season', 'week']].duplicated().sum())
print(qbs.head(20))

'''qb_name = "P.Mahomes"
qb_df = qbs[(qbs['player_name'] == qb_name) & (qbs['season'] == 2023)]

qb_df = qb_df.sort_values(['season', 'week'])

plt.figure(figsize=(10,5))
plt.plot(qb_df['week'], qb_df['passing_yards'], label='Actual Passing Yards', marker='o')
plt.plot(qb_df['week'], qb_df['passing_yards_roll3_mean'], label='3-Game Rolling Mean', marker='x')
plt.title(f"{qb_name} - Passing Yards vs 3-Game Rolling Mean")
plt.xlabel("Week")
plt.ylabel("Passing Yards")
plt.legend()
plt.grid(True)
plt.show()

qb_df[['season', 'week', 'passing_yards', 'passing_yards_roll3_mean']].head(10) '''

(2061, 49)
0
     player_id player_name  season season_type  week recent_team  \
0   00-0019596     T.Brady    2021         REG     1          TB   
1   00-0019596     T.Brady    2021         REG     2          TB   
2   00-0019596     T.Brady    2021         REG     3          TB   
3   00-0019596     T.Brady    2021         REG     4          TB   
4   00-0019596     T.Brady    2021         REG     5          TB   
5   00-0019596     T.Brady    2021         REG     6          TB   
6   00-0019596     T.Brady    2021         REG     7          TB   
7   00-0019596     T.Brady    2021         REG     8          TB   
8   00-0019596     T.Brady    2021         REG    10          TB   
9   00-0019596     T.Brady    2021         REG    11          TB   
10  00-0019596     T.Brady    2021         REG    12          TB   
11  00-0019596     T.Brady    2021         REG    13          TB   
12  00-0019596     T.Brady    2021         REG    14          TB   
13  00-0019596     T.Brady    2021 

'qb_name = "P.Mahomes"\nqb_df = qbs[(qbs[\'player_name\'] == qb_name) & (qbs[\'season\'] == 2023)]\n\nqb_df = qb_df.sort_values([\'season\', \'week\'])\n\nplt.figure(figsize=(10,5))\nplt.plot(qb_df[\'week\'], qb_df[\'passing_yards\'], label=\'Actual Passing Yards\', marker=\'o\')\nplt.plot(qb_df[\'week\'], qb_df[\'passing_yards_roll3_mean\'], label=\'3-Game Rolling Mean\', marker=\'x\')\nplt.title(f"{qb_name} - Passing Yards vs 3-Game Rolling Mean")\nplt.xlabel("Week")\nplt.ylabel("Passing Yards")\nplt.legend()\nplt.grid(True)\nplt.show()\n\nqb_df[[\'season\', \'week\', \'passing_yards\', \'passing_yards_roll3_mean\']].head(10) '

In [179]:
# Check df to ensure to dupes or other errors
#print(qbs.sample(10))
#print(qbs[['player_id', 'season', 'week']].duplicated().sum())
#print(qbs['home_game'].value_counts(dropna=False))
'''qb_name = "P.Mahomes"
qb_df = qbs[(qbs['player_name'] == qb_name) & (qbs['season'] == 2023)]

plt.figure(figsize=(10,5))
plt.plot(qb_df['week'], qb_df['passing_yards'], label='Passing Yards', marker='o')
plt.plot(qb_df['week'], qb_df['passing_yards_roll3_mean'], label='3-Game Rolling Mean', marker='x')
plt.fill_between(qb_df['week'], 
                 qb_df['passing_yards_roll3_mean'] - qb_df['passing_yards_volatility'],
                 qb_df['passing_yards_roll3_mean'] + qb_df['passing_yards_volatility'],
                 alpha=0.2, label='Volatility Range')
plt.title(f"{qb_name} - Passing Yards, Rolling Mean & Volatility (2023)")
plt.xlabel("Week")
plt.ylabel("Passing Yards")
plt.legend()
plt.grid(True)
plt.show()'''

'qb_name = "P.Mahomes"\nqb_df = qbs[(qbs[\'player_name\'] == qb_name) & (qbs[\'season\'] == 2023)]\n\nplt.figure(figsize=(10,5))\nplt.plot(qb_df[\'week\'], qb_df[\'passing_yards\'], label=\'Passing Yards\', marker=\'o\')\nplt.plot(qb_df[\'week\'], qb_df[\'passing_yards_roll3_mean\'], label=\'3-Game Rolling Mean\', marker=\'x\')\nplt.fill_between(qb_df[\'week\'], \n                 qb_df[\'passing_yards_roll3_mean\'] - qb_df[\'passing_yards_volatility\'],\n                 qb_df[\'passing_yards_roll3_mean\'] + qb_df[\'passing_yards_volatility\'],\n                 alpha=0.2, label=\'Volatility Range\')\nplt.title(f"{qb_name} - Passing Yards, Rolling Mean & Volatility (2023)")\nplt.xlabel("Week")\nplt.ylabel("Passing Yards")\nplt.legend()\nplt.grid(True)\nplt.show()'